In [8]:
import time
import io
import cv2
import numpy
import re
import sqlite3
import dlib


from PIL import Image
from http.server import BaseHTTPRequestHandler, HTTPServer
from os import curdir
from os.path import join as pjoin
from scipy.spatial import distance
from random import choice
from string import ascii_letters
from string import digits



HOST_NAME = 'localhost' # !!!REMEMBER TO CHANGE THIS!!!
PORT_NUMBER = 1111 # Maybe set this to 9000.
DB_NAME = 'faces.db'

def searchInDb(desc):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    selectedData = []
    sel = cursor.execute("select id, desc from face")
    targetIndex = 0
    for line in sel:
        print(len(desc))
        if len(desc) != 0:
            euclidianDistanse = distance.euclidean(numpy.array(line[1].split('||'), dtype=float), numpy.array(desc.split('||'), dtype=float))
            if euclidianDistanse < 0.6:
                targetIndex = line[0]
                print(euclidianDistanse)
            else:
                print(str(euclidianDistanse) + '> 0.6')
    
    result = ''
    if targetIndex!=0:
        sel = cursor.execute("select rawstring from face where id=?", [targetIndex])
        for line in sel:
            result = line[0]    
    conn.close()
    
    return result

def create_key():
    key = "".join(choice(ascii_letters + digits) for i in range(20))
    return key

def appendToDb(keywords, rawstring, desc, image):
    if desc != '':
        conn = sqlite3.connect(DB_NAME)
        cursor = conn.cursor()
        sel = cursor.execute("SELECT key FROM face")
        sel_list = [i[0] for i in sel]
        key = create_key()
        if sel_list.count(key) > 1:
            while True:
                key = create_key()
                if sel_list.count(key) == 1:
                    break
        
        ins = cursor.execute("INSERT INTO face (rawstring, keywords, date, desc, key) VALUES( ?, ?, ?, ?, ?)",
                         (rawstring, keywords[0], keywords[1], desc, key))
        conn.commit()
        conn.close()
        image_name = 'images\\%s.png' % key 
        print(image_name)
        cv2.imwrite(image_name, image)

def getDescriptor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detector = dlib.get_frontal_face_detector()
    dets = detector(gray, 1)
    sp = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
    facerec = dlib.face_recognition_model_v1('dlib_face_recognition_resnet_model_v1.dat')
    faceDescriptor = ''
    for i, d in enumerate(dets):
            shape = sp(img, d)      
            faceDescriptor += str(facerec.compute_face_descriptor(img, shape)).replace('\n', '||')
            if i != len(dets) - 1:
                faceDescriptor += '\n'
    return faceDescriptor

def parseDataFromPost(stringToParse):
    keywords = ""
    date = ""
    splitted = stringToParse.split('\\\\row\\\\')
    for line in splitted:
        if line != '':
            keywords += line.split('=')[1] + '||' 
            if line.split('=')[0] == "Дата регистрации":
                date = line.split('=')[1]
    print(date)
    return keywords, date

def countOfRowsInDB():
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    count = 0
    res = cursor.execute("SELECT COUNT(*) FROM face")
    for line in res:
        count = line
    conn.close()
    return count[0]

def searchingInDB(searchString):
    conn = sqlite3.connect(DB_NAME)
    cursor = conn.cursor()
    res = cursor.execute("SELECT keywords FROM face")
    for line in res:
        print(line)
    return(1)
    

class MyHandler(BaseHTTPRequestHandler):
    store_path = pjoin(curdir, '1.jpg')
    def _set_headers(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()

    def do_GET(self):
        self._set_headers()
        countRows = countOfRowsInDB()
        responseString = "||blk||count rows = " + str(countRows) + "||blk||"
        self.wfile.write(responseString.encode())
        

    def do_HEAD(self):
        self._set_headers()
        
    def do_POST(self):
        # Doesn't do anything with posted data
        self._set_headers()
        params = self.rfile.read(int(self.headers['Content-Length']))
        paramsSplit = params.split(b'\\image\\')
        method = paramsSplit[0].split(b'\\method\\')[0].decode()
        print(method)
       
        
        if method == 'appendToDb':
            dataString = paramsSplit[0].split(b'\\method\\')[1].decode()
            print(dataString)
            keywordsAndDate = parseDataFromPost(dataString)
            img = Image.open(io.BytesIO(paramsSplit[1]))
            image = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2RGB)
            desc = getDescriptor(image)
            appendToDb(keywordsAndDate, dataString, desc, image)
            self.wfile.write("Succsess POST".encode())
            
        elif method == 'searchInDb':
            imbyte = params.split(b'\\method\\')[1]
            img = Image.open(io.BytesIO(imbyte))
            image = cv2.cvtColor(numpy.array(img), cv2.COLOR_BGR2RGB)
            desc = getDescriptor(image)
            searched = searchInDb(desc)
            print(searched)
            sendingString = "Succsess POST\\content\\\\" + searched
            self.wfile.write(bytes(str(len(sendingString)) + "\\length\\" + sendingString, 'utf-8').decode('unicode-escape').encode('ISO-8859-1'))        
#             self.wfile.write((str(len(sendingString)) + "\\length\\" + sendingString).encode())
        elif method == 'fullTextSearch':
            print(str(params.decode()))
            searchString = str(params.decode()).split('<text>')[1]
            searchingInDB(searchString)
if __name__ == '__main__':
    server_class = HTTPServer
    httpd = server_class((HOST_NAME, PORT_NUMBER), MyHandler)
    print(time.asctime(), "Server Starts - %s:%s" % (HOST_NAME, PORT_NUMBER))
    try:
        httpd.serve_forever()
    except KeyboardInterrupt:
        pass
    httpd.server_close()
    print(time.asctime(), "Server Stops - %s:%s" % (HOST_NAME, PORT_NUMBER))

Tue Apr 17 20:36:16 2018 Server Starts - localhost:1111


127.0.0.1 - - [17/Apr/2018 20:36:19] "POST / HTTP/1.1" 200 -


searchInDb
1430
0.46441021423997425
1430
0.4911362532450144
1430
0.5022940660913724
1430
0.4870513944297735
1430
0.48209906949951564
1430
0.5336312721224648
1430
0.5318069569872137
1430
0.5157140995705822
1430
0.48526372819568653
\\row\\aaa=aaaa\\row\\Дата регистрации=15.04.2018 22:46:03
searchInDb


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:164: DeprecationWarning: invalid escape sequence '\l'
127.0.0.1 - - [17/Apr/2018 20:36:23] "POST / HTTP/1.1" 200 -


1420
0.37948839328685846
1420
0.3839913318554958
1420
0.4027094424377494
1420
0.4047460614506268
1420
0.367013315367535
1420
0.4410031703067779
1420
0.39949325129341257
1420
0.4067872140318336
1420
0.3953881528584721
\\row\\aaa=aaaa\\row\\Дата регистрации=15.04.2018 22:46:03
searchInDb


...

1414
0.43867020142401164
1414
0.4410418515025755
1414
0.4440891994529106
1414
0.44324498794031475
1414
0.396809407262809
1414
0.47933495524360986
1414
0.46787280262284714
1414
0.44566984169619983
1414
0.4294436476211985
\\row\\aaa=aaaa\\row\\Дата регистрации=15.04.2018 22:46:03
searchInDb


127.0.0.1 - - [17/Apr/2018 20:38:28] "POST / HTTP/1.1" 200 -


1423
0.3924473358632836
1423
0.3770775708624114
1423
0.4168841239365337
1423
0.40686121448765544
1423
0.3789779913535811
1423
0.44807921221972224
1423
0.42100640163775976
1423
0.4430075756780927
1423
0.40633467893136305
\\row\\aaa=aaaa\\row\\Дата регистрации=15.04.2018 22:46:03
searchInDb


127.0.0.1 - - [17/Apr/2018 20:38:32] "POST / HTTP/1.1" 200 -


1419
0.3576442733681288
1419
0.3542464876196344
1419
0.3733671875791038
1419
0.3459766397485248
1419
0.333288162280443
1419
0.40735873463091776
1419
0.36189572647959384
1419
0.38425562494661897
1419
0.36620540663092704
\\row\\aaa=aaaa\\row\\Дата регистрации=15.04.2018 22:46:03
Tue Apr 17 20:40:58 2018 Server Stops - localhost:1111
